# Find all the types of documents posible in supercias
### We need to define how many types documents and their frequency in supercias

In [1]:
packages = ['pymongo', 'gridfs', 'pandas', 'pymupdf', 'pytesseract', 'pdf2image', 'openai', 'python-dotenv']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
import pandas as pd
from functions.pdf_operations import PDFOperations
from functions.prompt_openai import query_openai
# endpoint = '10.0.10.5:27017' # use vpn
endpoint = '192.168.1.10:27017'
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'clean_companies'
# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# get collection
collection = db[collection]
# where the pdfs will be stored 
pdfs_path = '../storage/pdfs/'
# pdf operator
pdf_op = PDFOperations(db, 'companies', pdfs_path)
#find pdf file
companies_cursor = collection.find()


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Let seperate the docuemt by the type they can belong to: Generales, Economicos  and Judiciales

In [3]:
# now lets get all the documents for every company
companies = collection.find()
Documentos_generales = []
Documentos_economicos = []
Documentos_juridicos = []

for company in companies:
    # get the Documentos online
    documentos = company['Documentos']
    for documento in documentos['Generales']:
        Documentos_generales.append(documento)
    for documento in documentos['Economicos']:
        Documentos_economicos.append(documento)
    for documento in documentos['Juridicos']:
        Documentos_juridicos.append(documento)

Get all of the difrent kind of documentos generales into a df

In [4]:
def proces_documents(documentos, label_fields):
    docs = []
    for documento in documentos:
        fields = documento.split('.')[0].split('_')
        new_doc = {}
        for index, field in enumerate(fields):
            new_doc[label_fields[index]] = field
        # add filename at the end
        new_doc['filename'] = documento
        # add new doc
        docs.append(new_doc)
    return docs

doc_generales_fields = ['ruc', 'tipo_doc', 'documento', 'fecha', 'nombre', 'cargo', 'other']
docs = proces_documents(Documentos_generales, doc_generales_fields)
df_generales = pd.DataFrame(docs)
df_generales.head()

,ruc,tipo_doc,documento,fecha,filename,nombre,cargo
0,0791841030001,DocumentosGenerales,Formulario de actualización de datos,2022-08-04,0791841030001_DocumentosGenerales_Formulario d...,NaN,NaN
1,0791841030001,DocumentosGenerales,Formulario de actualización de datos,2024-04-02,0791841030001_DocumentosGenerales_Formulario d...,NaN,NaN
2,0791841030001,DocumentosGenerales,Nombramiento,2023-01-26,0791841030001_DocumentosGenerales_Nombramiento...,FAJARDO TINOCO ISAURO DE JESUS,PRESIDENTE
3,0791841030001,DocumentosGenerales,Nombramiento SAS,2022-08-02,0791841030001_DocumentosGenerales_Nombramiento...,FAJARDO FAJARDO LESLIE FABIOLA,GERENTE GENERAL
4,0791841030001,DocumentosGenerales,Formulario de actualización de datos,2023-08-07,0791841030001_DocumentosGenerales_Formulario d...,NaN,NaN


In [5]:
# get all the ditribution of the documents
print(df_generales['documento'].value_counts())

documento
Oficio Nombramiento Administradores                        166636
Formulario de actualización de datos                        86756
Oficio Transferencia Acciones                               83115
Nombramiento                                                43206
Razón de Inscripción registrada                             39293
Formulario de Información de la Sociedad Extranjera         26996
Certificado de Existencia Legal                             26987
Poder otorgado por la Sociedad Extranjera                   26539
Oficio de Presentación de Doc                               15612
Declaración Juramentada  Nómina de Socios o Accionistas     15405
Solicitud de acceso y declaración de responsabilidad        13575
Escritura Cesión Participación                              10067
Carta de autorización                                        7693
Notificación general                                         2696
Renuncia                                                     2321


Now we evtract the text from every possible Oficio Nombramiento Administradores

We need to define what data we want to extact from every Oficio Nombramiento Administradores

In [6]:
import concurrent.futures
import threading
import logging
import traceback

# Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(threadName)s - %(message)s')

# Create a global lock for printing (if needed)
print_lock = threading.Lock()

# create a collection for Oficio Nombramiento Administradores un cmongoDB
collection = db['Oficio_Nombramiento_Administradores']
# get all the documents which have the document 'Oficio Nombramiento Administradores' and have the date containes '2023'
df_oficio_nombramiento_administradores = df_generales[(df_generales['documento'] == 'Oficio Nombramiento Administradores')]
# extracted structure for 'Oficio Nombramiento Administradores'
struct = {
    "company": {
        "ruc": "string",
        "name": "string",
        "region": "string",
        "date": "string",
        "address": "string",
        "phone": "string",
        "notary": "string",
        "lawyer": "string",
    },
    "appoitment":{
        "position": "string",
        "term_years": "string",
        "appointment_date": "string",
        "previous_holder": {
            "name": "string",
            "designated_date": "string",
            "id_number": "string",
            "id_type": "string",
        },
        "new_holder": {
            "name": "string",
            "designation_date": "string",
            "id_number": "string",
            "id_type": "string",
        },
    },
}

def extract_data_from_pdf(row, collection):
    try: 
        # check if the file exists in the collection
        filename = row['filename']
        if collection.find_one({'filename': filename}):
            #print(f'{filename} already exists in the collection')
            return
        print(f'Extracting data from {filename}')
        text = pdf_op.extract_text(filename)
        # get the size of the text
        tokens = len(text.split())
        if tokens < 100:
            print(f'Text size: {tokens}')
            print(f'Not enough text in {filename}')
            return
        elif tokens > 16385:
            print(f'Text size: {tokens}')
            print(f'Too much text in {filename}')
            return
        # query open AI
        response = query_openai("extract the following data in the json format, with any N/A: " + str(struct) + " from the following text: " + text)
        response = eval(response)
        #response = {}
        response['ruc'] = row['ruc']
        response['fecha'] = row['fecha']
        response['nombre'] = row['nombre']
        response['cargo'] = row['cargo']
        response['filename'] = filename
        print(response)
        # add to the collection
        collection.insert_one(response)
        print('-----------------------------------')
    except Exception as e:
        print(f'Error extracting data from {filename}: {e}')


Run Sequentially

In [13]:
# test the first values of the df between 100 and 200
for _, row in df_oficio_nombramiento_administradores.iloc[12:2000].iterrows():
    extract_data_from_pdf(row, collection)

Extracting data from 1790721450001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-09-04_ALVERNIA DE CHACON IMELDA_PRESIDENTE.pdf
Error extracting data from 1790721450001_DocumentosGenerales_Oficio Nombramiento Administradores_2020-09-04_ALVERNIA DE CHACON IMELDA_PRESIDENTE.pdf: cannot identify image file <_io.BytesIO object at 0x7fa6e3da7a60>
Extracting data from 1791805348001_DocumentosGenerales_Oficio Nombramiento Administradores_2012-09-11_ZAMORA VEGA LUIS GUSTAVO_APODERADO.pdf
Text size: 215164


2024-10-05 21:03:04,421 - MainThread - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


Error extracting data from 1791805348001_DocumentosGenerales_Oficio Nombramiento Administradores_2012-09-11_ZAMORA VEGA LUIS GUSTAVO_APODERADO.pdf: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 68334 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Extracting data from 1391748027001_DocumentosGenerales_Oficio Nombramiento Administradores_2017-12-19_PALADINES LUISA MARLENE_GERENTE GENERAL.pdf
Error extracting data from 1391748027001_DocumentosGenerales_Oficio Nombramiento Administradores_2017-12-19_PALADINES LUISA MARLENE_GERENTE GENERAL.pdf: cannot identify image file <_io.BytesIO object at 0x7fa6e3dcb3d0>


Run multithreaded

In [ ]:
# Multithreading implementation
def process_pdf_data_in_parallel(df, collection, max_workers=5):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks for each row to the executor, passing the collection explicitly
        futures = [executor.submit(extract_data_from_pdf, row, collection) for _, row in df.iterrows()]
        # Optionally, you can handle results or exceptions here
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # We can use this to raise exceptions if any occurred in the threads
            except Exception as e:
                # remove row from df if error
                print(f"Exception in thread: {e}")
                traceback.print_exc()


# Adjust the number of threads as needed
max_threads = 10  # Example: Use 5 threads
# Execute the function with multithreading, passing the collection explicitly
process_pdf_data_in_parallel(df_oficio_nombramiento_administradores, collection, max_threads)

In [ ]:
# get all the ditribution of the documents economicosn 
print(df_economicos['documento'].value_counts())

In [ ]:
docs_economicos_fields = ['ruc', 'tipo_doc', 'documento', 'fecha', 'other']
docs = proces_documents(Documentos_economicos, docs_economicos_fields)
df_economicos = pd.DataFrame(docs)
df_economicos.head()

In [ ]:
docs_juridicos_fields = ['ruc', 'tipo_doc', 'acto_juridico', 'fecha', 'other']
docs = proces_documents(Documentos_juridicos, docs_juridicos_fields)
df_juridicos = pd.DataFrame(docs)
df_juridicos.head()

In [ ]:
# get all the ditribution of the documents economicosn 
print(df_juridicos['acto_juridico'].value_counts())

Now get all of the the documentos economicos

Check the pdfs for the pdfs that we don't have the complete title or filename for
this it important because based on the document name we define how we can extract the information 

In [ ]:
#print(f'In the df_generales there are {df_generales_nan.shape[0]} out of {df_generales.shape[0]} with no fecha column')
#df_generales['documento'].value_counts()

In [7]:

# make the same for economicos
#df_economicos['documento'].value_counts().shape

In [ ]:
df_juridicos_fecha_nan = df_juridicos[df_juridicos['fecha'].isna()]

df_juridicos_fecha_nan.shape

# get all docuemntos de 2023
#df_juridicos[df_juridicos['fecha'].str.contains('2023')].head()
#df_juridicos['acto_juridico'].value_counts().shape
# make the same for juridicos
#df_juridico_2023 = df_juridicos['acto_juridico'].value_counts()
